<a href="https://colab.research.google.com/github/Roghiehghahremani/Deep-Learning-With-PyTorch---Full-Course_Patrick-Loeber/blob/main/15_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ImageFolder
# Scheduler
# Transfer learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets,models,transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/DLPyTorch_Patrick_Loeber

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/DLPyTorch_Patrick_Loeber


In [11]:
mean = np.array([0.485, 0.456,0.406])
std  = np.array([0.229, 0.224, 0.225])

data_transforms ={
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
    'val':transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
}

In [6]:
# import data
data_dir = 'data/hymenoptera_data'
sets = ['train','val']

image_datasets = {x:datasets.ImageFolder(os.path.join(data_dir,x), data_transforms[x]) for x in ['train', 'val']}

dataloaders  = {x:torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0) for x in ['train', 'val']}

dataset_sizes ={x: len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(class_names)

['ants', 'bees']


In [7]:
def train_model(model,criterion,optimizer,scheculer, num_epochs=25):
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print(f' Epoch {epoch} / {num_epochs-1}')
    print('-'*10)

    # each epoch has training and validation phase
    for phase in ['train','val']:
      if phase == 'train':
        model.train() # set model to training mode
      else:
        model.eval() # set model to validation mode

      running_loss = 0.0
      running_corrects = 0

      # Iterator over data
      for inputs,labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        #forward
        # track history if you only in train
        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(inputs)
          _,preds = torch.max(outputs,1)
          loss = criterion(outputs,labels)

          # backward plus optimizer only if you are in training phase
          if phase == 'train':
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # statistics
        running_loss +=loss.item()*inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

      if phase == 'train':
        scheduler.step()

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects.double() / dataset_sizes[phase]

      print(f'{phase} loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

      # deep copy the model
      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())

    print()
  time_elapsed = time.time() - since
  print(f' training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
  print(f'best val acc: {best_acc:4f}')

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model

In [14]:
# first option

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs,2)
model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr = 0.001)

# scheduler

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7,gamma=0.1)

model = train_model(model,criterion,optimizer,step_lr_scheduler,num_epochs=2)




NameError: name 'scheduler' is not defined

In [ ]:
# second option

model = models.resnet18(pretrained=True)

for param in model.parameters():
  param.requires_grad = False

num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs,2)
model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr = 0.001)

# scheduler

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7,gamma=0.1)

model = train_model(model,criterion,optimizer,step_lr_scheduler,num_epochs=20)